Student name <br> Faiza Anan Noor 
Student number 2306676 <br>
02, 07, 2024  <br>

# Exercise 2 | TKO_7092 Evaluation of Machine Learning Methods 2024

### Prediction of the metal ion content from multi-parameter data
<b> Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), using number of neighbors k = 1, 3, 5, 7.</b> <br>

<b> Instructions: </b> 

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score (using sklearn.preprocessing.StandardScaler is advised).
    - Implement Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Explain your code by adding detailed comments. 
    - Only provide code that is relevant to the exercise.
    - Please submit your solution as a Jupyter Notebook (.ipynb) and as a PDF file. Ensure to include your full name in the filename.     
    - Submit to moodle your solution on ** Wednesday 7 of February ** at the latest.
  
<b>Please follow the instructions and note that you are expected to submit your individual solution.<br>
Identical or overly similar submissions will result in the exercise being marked as failed.
</b>

## Import libraries

In [1]:
# In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

## Read and visualize the dataset

In [2]:
# In this cell read the file Water_data.csv
# Print the dataset dimesions (i.e. number of rows and columns)
# Print the first 5 rows of the dataset

df=pd.read_csv("water_data.csv")
df.head(5)

,Mod1,Mod2,Mod3,c_total,Cd,Pb
0,9945,119,72335,0,0.0,0.0
1,9596,119,110542,0,0.0,0.0
2,10812,120,98594,0,0.0,0.0
3,10786,117,82977,0,0.0,0.0
4,10566,108,136416,14,0.0,14.0


In [3]:
print("No. of rows and columns is ",df.shape)

No. of rows and columns is  (268, 6)


## Standardization of the dataset

 In this cell, we standardize the dataset features by removing the mean and scaling to unit variance. 
 In other words, we used z-score to scale the dataset features (Mod1, Mod2, Mod3) using StandardScaler() and fit and transformed the dataFrame and stored the updated one in df_standardized. Then we printed the 5 first samples (i.e. rows) of the scaled dataset


In [4]:
# In this cell, standardize the dataset features by removing the mean and scaling to unit variance. 
# In other words, use z-score to scale the dataset features (Mod1, Mod2, Mod3) 
# Print the 5 first samples (i.e. rows) of the scaled dataset

scaler = StandardScaler()

X_subset=df[['Mod1','Mod2','Mod3']]  
y_subset = df[['c_total', 'Cd', 'Pb']]

# Fit and transform the DataFrame
X_standardized= pd.DataFrame(scaler.fit_transform(X_subset), columns=X_subset. columns)
df_standardized=pd.concat([X_standardized, y_subset], axis=1)

# Now df_standardized contains the standardized features using Z-score normalization

X = df_standardized[['Mod1','Mod2','Mod3']].values  
y = df_standardized[['c_total', 'Cd', 'Pb']].values
df_standardized.head(5)

,Mod1,Mod2,Mod3,c_total,Cd,Pb
0,-0.972283,-0.670482,-0.358179,0,0.0,0.0
1,-0.975878,-0.670482,0.259488,0,0.0,0.0
2,-0.963351,-0.670394,0.066333,0,0.0,0.0
3,-0.963619,-0.670657,-0.186137,0,0.0,0.0
4,-0.965885,-0.671447,0.677776,14,0.0,14.0


## C-index code 

In [5]:
# In this cell, implement the C-index function. You may use the implementation from the first exercise.

def cindex(y, yp):
    n=0
    #n = 0
    h_num = 0 
    for i in range(0, len(y)):
        t = y[i]
        p = yp[i]
        for j in range(i+1, len(y)):
            nt = y[j]
            np = yp[j]
            if (t != nt): 
                n = n + 1
                if (p < np and t < nt) or (p > np and t > nt): 
                    h_num += 1
                elif (p == np):
                    h_num += 0.5
   
    return h_num/n

In [ ]:
# Test the cindex function with following values
true_labels = np.array([-1, 1, 1, -1, 1])
predictions = np.array([0.60, 0.80, 0.75, 0.75, 0.70])
print(predictions.shape)
cindx = cindex(true_labels, predictions)
print(cindx) #For this example, a correct C-index implementation will result in 0.75

## Leave-One-Out cross-validation
In the following cell, write and execute your code for Leave-One-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-One-Out C-index for c_total, Cd and Pb for each k value.


***
We implemented the Leave-One-Out cross validation from scratch by manually deleting just one index  from the targets and labels for training to be evaluated later on.
Then for different values of k=1,3,5,7 we ran K nearest Neighbors using these k values and stored the c_total, Pb, Pb cindex for each of them. After each iteration, for each k value also,  we stored the predictions and answers for C_total, Pb, Cd, and then after the iteration we stored cindex for each of these variables by using their Real(Answers) and Predicted Values by also adopting the necessary shaping anf flattening. We stored the Cindices in arrays to be used later on for plotting.

In [ ]:

# Defining variables and arrays for storing the cindex for c_total, Pb, Cd and arrays also to store them for plotting later
avg_c_total_loo=0
avg_c_cd_loo=0
avg_c_pb_loo=0

avg_c_total_loo_total=[]
avg_c_cd_loo_total=[]
avg_c_pb_loo_total=[]


# Define k values
k_values = [1, 3, 5, 7]

# Perform Leave-One-Out cross-validation
for k in k_values:
    # Defining arrays for storing answers and predictions which contain all 3 variables at once(c_total, Cd, Pb)
    # These will be used later on by referring to indices 0, 1, 2 to retrieve answers and predictions for c_total, Cd, Pb
    # and then cindex will be found out for each of them using these answers and predictions for each variable and we also did necessary indices
    answers = []
    predictions=[]
    for i in range(len(X)):
       # predictions = []
        # Split data into train and test sets (leave one out manually)
        # Define training and testing sets using boolean indexing
        X_train = np.delete(X, i, axis=0)  
        # Leave out one. we delete specific index from training to be tested on later for targets and labels both
        y_train = np.delete(y, i, axis=0)
        
        
        # Create test set with only the current index
        X_test = X[i].reshape(1, -1)
        y_test = y[i].reshape(1, -1)
        
        # Train KNN regression model
        knn_model = KNeighborsRegressor(n_neighbors=k)
        knn_model.fit(X_train, y_train)
        
        # Predict on test set containing the ith feature
        y_pred = knn_model.predict(X_test)
        # Storing the predictions and answers by appropriate shaping
        predictions.append(y_pred.reshape(-1).tolist())
        
        answers.append(y_test.reshape(-1).tolist())
     
    
    # Calculate C-index for each output
    # 0 index refers to c_total, 1 refers to Cd, 2 refers to Pb
    avg_c_total_loo=cindex([ans[0] for ans in answers], [pred[0] for pred in predictions])
    avg_cd_loo=cindex([ans[1] for ans in answers], [pred[1] for pred in predictions])
    avg_pb_loo=cindex([ans[2] for ans in answers], [pred[2] for pred in predictions])
    
    # Storing the respective cindices in arrays for later use(plotting)
    
    avg_c_total_loo_total.append(avg_c_total_loo)
    avg_c_cd_loo_total.append(avg_cd_loo)
    avg_c_pb_loo_total.append(avg_pb_loo)


    # Print results
    print("K = ",k)
    print(f"Average C-index for c_total: {avg_c_total_loo: .3f}")    
    print(f"Average C-index for Cd: {avg_cd_loo: .3f}")
    
    print(f"Average C-index for Pb: {avg_pb_loo: .3f}")
    


## Leave-Replicas-Out cross-validation

In the following cell, write and execute your code for Leave-Replicas-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-Replicas-Out C-index for c_total, Cd and Pb for each k value.

In [ ]:
y1 =df_standardized["c_total"].values # Array of c-total values
y2 = df_standardized["Cd"].values   # Array of Cd values
y3 = df_standardized["Pb"].values   # Array of Pb values
y = df_standardized[['c_total', 'Cd', 'Pb']].values

X = df_standardized[['Mod1','Mod2','Mod3']].values

Using sklearn.model_selection, we carried out Leave-Replicas-Out cross-validation. With the use of LeaveOneGroupOut, a cross-validator, data may be divided into training sets with all samples excluded from one particular group by providing train/test indices.  The first step in grouping the replicas accurately is to identify the groups inside the dataset; that is, each replica should have identical values for c-total, Cd, and Pb.

In [ ]:
# Group replicas with equal c-total, Cd, and Pb values
group_indices = {}
for i, (c_total, cd, pb) in enumerate(zip(y1, y2, y3)):
    key = (c_total, cd, pb)
    if key not in group_indices:
        group_indices[key] = []
    group_indices[key].append(i)

# Convert dictionary values to list of tuples
groups = list(group_indices.values())
groups[:5]

Group labels for the samples used while splitting the dataset into train/test set. This ‘groups’ parameter must always be specified to calculate the number of splits. We calculate this using the group indices we defined earlier

In [ ]:
# Create an array of group labels for LeaveOneGroupOut cross-validation
group_labels = np.zeros(len(X))
for idx, group_indices in enumerate(groups):
    group_labels[group_indices] = idx

Then for different values of k=1,3,5,7 we ran K nearest Neighbors using these k values and stored the c_total, Pb, Pb cindex for each of them. After each iteration, for each k value also,  we stored the predictions and answers for C_total, Pb, Cd, and then after the iteration we stored cindex for each of these variables by using their Real(Answers) and Predicted Values by also adopting the necessary shaping and flattening. We stored the Cindices in arrays to be used later on for plotting.

In [ ]:
#aRRAYS TO STORE CINDICES FOR C_TOTAL, cd, Pb
avg_c_total_lro_list=[]
avg_cd_lro_list=[]
avg_pb_lro_list=[]

# Define k values
k_values = [1, 3, 5, 7]


# Perform Leave-One-Out cross-validation
for k in k_values:
    
    # defining arrays for all answers, predictions for all variables(C_total, Pb, Cd)
    # and also 3 different pairs arrays for storing specific results for just c_total, Pb, Cd
    
    answers=[]
    predictions=[]
    
    predictions_ctotal=[]
    answers_ctotal=[]
    predictions_Cd=[]
    answers_Cd=[]
    predictions_Pb=[]
    answers_Pb=[]


    
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, groups=group_labels):
        
        # Forming Train and test sets using the indices
        X_train, X_test = X[train_index], X[test_index]        
        y_train, y_test = y[train_index], y[test_index]
        
        # Fitting the model using the specified value of k
               
        knn_model = KNeighborsRegressor(n_neighbors=k)
        knn_model.fit(X_train, y_train)
        y_pred = knn_model.predict(X_test)
        
        # Storing actual answers and predictions which tontain values for all variables: c_total, Cd, Pb. 
        #This will be used to extract results(predictions and answers) for each of them in the upcoming code snippets
        predictions.append(y_pred)
        answers.append(y_test)
        
        # Appending C_total real(answers) and results(answers) by using the appropriate index 
        predictions_ctotal.append(y_pred[:,0].flatten())
        answers_ctotal.append(y_test[:,0].flatten())
        
        # Appending Cd real(answers) and results(answers) by using the appropriate index 

        predictions_Cd.append(y_pred[:,1].flatten())
        answers_Cd.append(y_test[:,1].flatten())
        
        # Appending Pb real(answers) and results(answers) by using the appropriate index 

        predictions_Pb.append(y_pred[:,2].flatten())
        answers_Pb.append(y_test[:,2].flatten())



    # Print results
    print("K = ",k)
    avg_c_total_lro_list.append(cindex(np.array(answers_ctotal).flatten(), np.array(predictions_ctotal).flatten()))
    avg_cd_lro_list.append(cindex(np.array(answers_Cd).flatten(), np.array(predictions_Cd).flatten()))
    avg_pb_lro_list.append(cindex(np.array(answers_Pb).flatten(), np.array(predictions_Pb).flatten()))
    
    print(f"Average C-index for c_total: {cindex(np.array(answers_ctotal).flatten(), np.array(predictions_ctotal).flatten()): .5f}")
    print(f"Average C-index for Cd: {cindex(np.array(answers_Cd).flatten(), np.array(predictions_Cd).flatten()): .5f}")
    print(f"Average C-index for Pb: {cindex(np.array(answers_Pb).flatten(), np.array(predictions_Pb).flatten()): .5f}")

    print("     ")

## Plot Leave-One-Out and Leave-Replicas-Out Results

Note: You may plot the results as they were presented in the video lecture (refer to MOOC2-Module 2 .pptx slides).

In [ ]:

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plot c_total results
axs[0].plot(k_values, avg_c_total_loo_total, marker='o', label='LOO')
axs[0].plot(k_values, avg_c_total_lro_list, marker='o', label='LRO')
axs[0].set_title('C-Index vs k Value for c_total')
axs[0].set_xlabel('k Value')
axs[0].set_ylabel('C-Index')
axs[0].legend()

# Plot Cd results
axs[1].plot(k_values, avg_c_cd_loo_total, marker='o', label='LOO')
axs[1].plot(k_values, avg_cd_lro_list, marker='o', label='LRO')
axs[1].set_title('C-Index vs k Value for Cd')
axs[1].set_xlabel('k Value')
axs[1].set_ylabel('C-Index')
axs[1].legend()

# Plot Pb results
axs[2].plot(k_values, avg_c_pb_loo_total, marker='o', label='LOO')
axs[2].plot(k_values, avg_pb_lro_list, marker='o', label='LRO')
axs[2].set_title('C-Index vs k Value for Pb')
axs[2].set_xlabel('k Value')
axs[2].set_ylabel('C-Index')
axs[2].legend()

# Set x-axis range from 0 to 1
for ax in axs:
    ax.set_ylim(0, 1)
    ax.set_xlim(1, 7)  # Set y-axis limit to specific values

# Show plot
plt.tight_layout()
plt.show()


## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation method had more optimistic results? 
- Explain the reason for the optimistic results produced by the cross-validation method.
- Which cross-validation method generalized better on unseen data? Why?

In [ ]:
#In this cell write your answers to the questions.

#### Which cross-validation method had more optimistic results? 
Leave-One-Out (LOO) cross-validation frequently produces more optimistic findings than Leave-Replicas-Out (LRO) cross-validation like in our case.

#### Explain the reason for the optimistic results produced by the cross-validation method

For each sample in the dataset, LOO cross-validation entails training the model on n −1 samples and testing it on the final sample. LRO cross-validation, on the other hand, separates the data into groups or replicas and makes sure the model is tested and trained on various subsets of the data. 

Because of the remarkably identical training sets that Leave-One-Out (LOO) cross-validation creates, it frequently yields more optimistic findings than Leave-Replicas-Out (LRO) cross-validation. With the exception of one sample, every training set in LOO is almost exactly the same as the whole dataset. Due to the fact that it essentially saw a similar sample during training, this high similarity might create overfitting, in which the model performs well on the test set. By splitting the data into separate replicas, LRO cross-validation, on the other hand, makes sure that the model is tested and trained on diverse subsets of the data. Less optimistic findings than with LOO are obtained because this diversity in training sets reduces overfitting and gives a more accurate assessment of model performance.

Because LOO uses all available data for training, it may thus seem promising at first, but compared to LRO, which offers a more reliable evaluation of model generalization, LOO frequently yields excessively optimistic findings.

#### Which cross-validation method generalized better on unseen data? Why?


When it comes to generalizing to new data, Leave-Replicas-Out (LRO) cross-validation performs better than Leave-One-Out (LOO) cross-validation in most cases. In contrast to the extremely similar training sets in LOO, LRO provides a more diversified training set, which helps minimize overfitting. Furthermore, because LRO guarantees testing on unseen copies during training, it is less impacted by data replication bias and produces estimates of model performance on unseen data that are more accurate. By dividing the data into groups or replicas, LRO cross-validation makes sure that the model is tested and trained on various subsets of the data. This variance in training sets yields a more accurate assessment of model performance and reduces overfitting.



